In [203]:
import json, os

import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import plotly.express as px

from sklearn.manifold import MDS
from sklearn.neighbors import KNeighborsClassifier

In [2]:
%matplotlib qt

### Récupération des données

In [3]:
rootdir ='C:/Users/hisma/Documents/projets_perso/votes/votes/'
content=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        f=open(os.path.join(subdir,file),'r')
        content.append(json.load(f))
        f.close()

In [4]:
rootdir ='C:/Users/hisma/Documents/projets_perso/votes/acteurs/acteur/'
acteurs_raw=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        f=open(os.path.join(subdir,file),'r')
        acteurs_raw.append(json.load(f))
        f.close()

acteurs={}
for acteur in acteurs_raw:
    acteurs[acteur['acteur']['uid']['#text']]=acteur

In [5]:
rootdir ='C:/Users/hisma/Documents/projets_perso/votes/acteurs/organe/'
organes_raw=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        f=open(os.path.join(subdir,file),'r')
        organes_raw.append(json.load(f))
        f.close()
        
organes={}
for organe in organes_raw:
    organes[organe['organe']['uid']]=organe

In [6]:
def add_vote(scrutin,votant,vote,d={}):
    if votant not in d.keys():
        d[votant]={}
    d[votant][scrutin]=vote
    
votes={}
for scrutin in content:
    for groupe in scrutin['scrutin']['ventilationVotes']['organe']['groupes']['groupe']:
        for vote,votants in groupe['vote']['decompteNominatif'].items():
            if votants is None:
                pass
            elif type(votants['votant'])==dict:
                add_vote(scrutin['scrutin']['uid'],votants['votant']['acteurRef'],vote[:-1],votes)
            else:
                for parlementaire in votants['votant']:
                    add_vote(scrutin['scrutin']['uid'],parlementaire['acteurRef'],vote[:-1],votes)

for acteur in votes.keys():
    if acteur in acteurs.keys():
        uid=acteurs[acteur]['acteur']['uid']['#text']
        metier=acteurs[acteur]['acteur']['profession']['libelleCourant']
        catsocpro=acteurs[acteur]['acteur']['profession']['socProcINSEE']['catSocPro']
        famsocpro=acteurs[acteur]['acteur']['profession']['socProcINSEE']['famSocPro']
        nom,prenom=acteurs[acteur]['acteur']['etatCivil']['ident']['nom'],acteurs[acteur]['acteur']['etatCivil']['ident']['prenom']
        naissance=acteurs[acteur]['acteur']['etatCivil']['infoNaissance']['dateNais']
        for mandat in acteurs[acteur]['acteur']['mandats']['mandat']:
            if mandat['typeOrgane']=='PARPOL':
                parti_uid = mandat['organes']['organeRef']
                votes[acteur]['parti']=organes[parti_uid]['organe']['libelle']
        if type(metier)==str:
            votes[acteur]['metier']=metier
        else:
            votes[acteur]['metier']='Sans profession déclarée'
        if type(catsocpro)==str:
            votes[acteur]['catsocpro']=catsocpro
        else:
            votes[acteur]['catsocpro']='Sans profession déclarée'
        if type(famsocpro)==str:
            votes[acteur]['famsocpro']=famsocpro
        else:
            votes[acteur]['famsocpro']='Sans profession déclarée'
        votes[acteur]['uid']=uid
        votes[acteur]['nom']=nom
        votes[acteur]['prenom']=prenom
        votes[acteur]['naissance']=naissance
        votes[acteur]
    else:
        pass


### Formation du tableau

In [7]:
df=pd.DataFrame(votes).transpose()
df=df[df['parti'].notna()] #570/577
df.reset_index(inplace=True)
effectifs_parti=df[['index','parti']].groupby('parti').count()
df=df.replace(effectifs_parti[effectifs_parti['index']<3].index,'Autre')
df.sort_values('parti')
df=df.reset_index(drop=True)
df=df.rename(columns={'index':'acteur_id'})
df=df.replace('nonVotant',np.nan)
X=df.drop(columns=['acteur_id','uid','nom','prenom','naissance','parti','metier','catsocpro','famsocpro'])
y=df[['acteur_id','uid','nom','prenom','naissance','parti','metier','catsocpro','famsocpro']]

### Matrice de similarité

In [219]:
"""
W=np.zeros((len(X),len(X)))
problems=[]
for i in range(len(X)-1):
    if i%10==0:
        print(i)
    for j in range(i+1,len(X)):
        denominator=(X.iloc[i].notna() & X.iloc[j].notna()).sum()
        if denominator==0: #We might need to delete a few absent guys
            problems.append([i,j])
        else:
            W[i][j]=1-(((X.iloc[i]==X.iloc[j]).sum())/denominator)
W[problems]=W.mean()
W+=W.T #to get a symetric matrix


np.save('dissimilarity',W)
"""


W=np.load('dissimilarity.npy')

In [11]:
#plt.imshow(W)

#### Attendance

In [9]:
attendance=X.notna().sum(axis='columns')
y.loc[:,'assiduite']=attendance

C:\Users\hisma\.conda\envs\Mds\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


### Embedding

Les axes n'ont pas de sens mathématiques, j'ai délibéremment décalé le centre, et fait tourner le graphique pour (très) approximativement match l'[échiquier politique](https://fr.wikipedia.org/wiki/%C3%89chiquier_politique)

In [257]:
"""
embedding=MDS(n_components=2,dissimilarity='precomputed')
positions=embedding.fit_transform(W)

rotate_by=0.4*np.pi
x_intercept=0.1
y_intercept=-0.15

before_x = positions[:,0]-positions[:,0].mean()
before_y = positions[:,1]-positions[:,1].mean()
new_x=np.cos(rotate_by)*before_x - np.sin(rotate_by)*before_y
new_y=np.sin(rotate_by)*before_x + np.cos(rotate_by)*before_y
new_positions=np.column_stack((new_x+x_intercept,new_y+y_intercept))

np.save('embedding',new_positions)
"""

positions = np.load('embedding.npy')

res=pd.DataFrame(positions,columns=['X','Y'])
to_show=pd.concat((y,res),axis='columns')

### Représentation

In [258]:
colors = {'La République en Marche':"#f7c81e", 'Europe Écologie Les Verts':"#1cde07", 'Autre':"#919491",
           'Les Républicains':'#00b6e8', 'Parti socialiste':'#f705ab', 'Parti radical de gauche': '#b0040f',
           'Debout la France': '#02e1fa', 'Rassemblement national':'#025dfa', 'La France Insoumise' : '#c75255',
           'Parti communiste français': '#ff0008', 'Union des démocrates, radicaux et libéraux':'#9d04de',
            'Mouvement Démocrate': '#de8e04', 'Régions et peuples solidaires':'#000000'}

fig = px.scatter(to_show, x="X", y="Y", color='parti', color_discrete_map=colors,
                 hover_name="parti",
                 hover_data={'acteur_id' : False,
                             'uid' : False,
                             'X' : False,
                             'Y' : False,
                             'parti' : False,
                             'assiduite' : False,
                             'Métier' : to_show['metier'],
                             'Nom': to_show['prenom']+' '+to_show['nom'],
                             'Nombre de votes': to_show['assiduite']
                            },
                 size='assiduite',
                 labels={'parti':"Parti politique"},
                 width=1000, height=600)

fig.show()
fig.write_html("embedding_parti.html")

In [231]:
colors = {'Cadres et professions intellectuelles supérieures':"#007bc2",
          'Employés':"#fa4b9d",
          'Retraités':"#0a02a8",
          'Professions Intermédiaires':"#c04bfa",
          "Artisans, commerçants et chefs d'entreprise":"#ff9100",
          "Sans profession déclarée":"#919491",
          "Agriculteurs exploitants":"#9c570e",
          "Autres personnes sans activité professionnelle":"#919491",
          "Ouvriers":"#b50006",
          np.nan:"#919491"
          }

fig = px.scatter(to_show, x="X", y="Y", color='famsocpro', color_discrete_map=colors,
                 hover_name="metier",
                 hover_data={'acteur_id' : False,
                             'uid' : False,
                             'X' : False,
                             'Y' : False,
                             'parti' : False,
                             'assiduite' : False,
                             'famsocpro': False,
                             'Nom': to_show['prenom']+' '+to_show['nom'],
                             'Parti': to_show['parti'],
                             'Nombre de votes': to_show['assiduite']
                            },
                 size='assiduite',
                 labels={'famsocpro': "Famille socio-professionnelle de l'INSEE"},
                 width=1000, height=600)

fig.show()
fig.write_html("embedding_metier.html")

### Assiduité

Pas très représentatif, correspond à l'assiduité en fonction du parti.

In [232]:
fig,ax=plt.subplots()
sns.barplot(data=y,x='parti',y='assiduite',ax=ax)
ax.tick_params(labelrotation=45)
plt.tight_layout()

### Classification gauche / droite

Rien de nouveau, et puis subjectif

In [244]:
model = {'La République en Marche' : 1, 'Europe Écologie Les Verts' : -1, 'Autre' : -1,
           'Les Républicains' : 1, 'Parti socialiste' : 0, 'Parti radical de gauche': 0,
           'Debout la France': 1, 'Rassemblement national': 1, 'La France Insoumise' : 0,
           'Parti communiste français': 0, 'Union des démocrates, radicaux et libéraux': -1,
            'Mouvement Démocrate': 1, 'Régions et peuples solidaires':-1}
        # Droite = 1, Gauche = 0
y_gd=y['parti'].replace(model)
X_train=res[y_gd!=-1]
y_train=y_gd[y_gd!=-1]


knn=KNeighborsClassifier(n_neighbors=18)
knn.fit(X=X_train,y=y_train)
precision=200
xvalues=np.linspace(-0.55,0.65,precision)
yvalues=np.linspace(-0.7,0.7,precision)
xx,yy=np.meshgrid(xvalues,yvalues)
z=knn.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

In [245]:
colors = {'La République en Marche':"#f7c81e", 'Europe Écologie Les Verts':"#1cde07", 'Autre':"#919491",
           'Les Républicains':'#00b6e8', 'Parti socialiste':'#f705ab', 'Parti radical de gauche': '#b0040f',
           'Debout la France': '#02e1fa', 'Rassemblement national':'#025dfa', 'La France Insoumise' : '#c75255',
           'Parti communiste français': '#ff0008', 'Union des démocrates, radicaux et libéraux':'#9d04de',
            'Mouvement Démocrate': '#de8e04', 'Régions et peuples solidaires':'#000000'}

fig = px.scatter(to_show, x="X", y="Y", color='parti', color_discrete_map=colors,
                 hover_name="parti",
                 hover_data={'acteur_id' : False,
                             'uid' : False,
                             'X' : False,
                             'Y' : False,
                             'parti' : False,
                             'assiduite' : False,
                             'Métier' : to_show['metier'],
                             'Nom': to_show['prenom']+' '+to_show['nom'],
                             'Nombre de votes': to_show['assiduite']
                            },
                 size='assiduite',
                 labels={'parti':"Parti politique"},
                 width=1000, height=600)

fig.add_contour(x=xvalues,y=yvalues,z=z,
                        colorscale=[[0,'#e60054'],[1,'#0063f7']],
                        showscale=None,
                        showlegend=None,
                        hoverinfo='skip')

fig.show()
fig.write_html("test.html")

# 3D

A l'air de régler les 3 de gauches qui apparaissent à droite 

In [260]:
embedding=MDS(n_components=3,dissimilarity='precomputed')
positions_3D=embedding.fit_transform(W)
res_3D=pd.DataFrame(positions_3D,columns=['X','Y','Z'])
to_show_3D=pd.concat((y,res_3D),axis='columns')

In [261]:
colors = {'La République en Marche':"#f7c81e", 'Europe Écologie Les Verts':"#1cde07", 'Autre':"#919491",
           'Les Républicains':'#00b6e8', 'Parti socialiste':'#f705ab', 'Parti radical de gauche': '#b0040f',
           'Debout la France': '#02e1fa', 'Rassemblement national':'#025dfa', 'La France Insoumise' : '#c75255',
           'Parti communiste français': '#ff0008', 'Union des démocrates, radicaux et libéraux':'#9d04de',
            'Mouvement Démocrate': '#de8e04', 'Régions et peuples solidaires':'#000000'}

fig = px.scatter_3d(to_show_3D, x="X", y="Y", z="Z", color='parti', color_discrete_map=colors,
                 hover_name="parti",
                 hover_data={'acteur_id' : False,
                             'uid' : False,
                             'X' : False,
                             'Y' : False,
                             'Z' : False,
                             'parti' : False,
                             'assiduite' : False,
                             'Métier' : to_show_3D['metier'],
                             'Nom': to_show_3D['prenom']+' '+to_show_3D['nom'],
                             'Nombre de votes': to_show_3D['assiduite']
                            },
                 size='assiduite',
                 labels={'parti':"Parti politique"},
                 width=1000, height=600)

fig.show()
fig.write_html("embedding_3D.html")